# Language Modeling, RNN Hypothesis Function

### Introduction

In this lesson, we'll explore a typical problem in natural language processing: language modeling.  Once we explore the problem of language modeling, we'll see recurrent neural networks can be used to tackle this problem. 

### Language Modeling

Let's say that we are presented with the following sentence: 

* "The dog jumped over the ____."

The next word could be "fence" or "wall", or any number of words.  This is an example of language modeling.

In language modeling, given a sequence of words, we predict which word comes next.  More formally, we can state the problem of language modeling as the following:

> Given a sequence of words, $x_1, x_2, ... x^{(t)}$, compute the probability distribution of the next word $x^{(t + 1)}$.

* $P(x^{t + 1} | x ^{t}, ... x^{(1)})$

> Where $x^{(t + 1)}$ can be any word in the vocabulary $V = \{w_1 ... w_{|v|} \}$

So with the above formulation, one way to think of language modeling is as a classification task, where each word in the vocabulary is a potential outcome.

### The importance of language modeling

Language modeling is behind a lot of important tasks in natural language processing.  For example, the autocomplete task on a phone is a direct use of language modeling.  

But language modeling is also useful because if we can predict the next word in a sentence, it means that our model has likely capturing the meaning of a sentence.  And if we can accomplish that, then we can perhaps categorize a document -- whether our task is classify the sentiment, or the topic, or the author of the text.  

### A First Attempt at Language Modeling

As a first attempt, we could try an embedding, followed by a vanilla neural network.  Doing so, would look like the following: 

* document = "the dog jumped over"
* numericalized_doc = `[1, 10, 20, 6]`
* embedded_text = $[e_{1x100}, e_{2x100}, e_{3x100}, e_{4x100}]$
* $Z = W_{i, 4*100}*e + b$
* $H = softmax(Z)$

* document = "dog jumped over the"
* numericalized_doc = `[10, 20, 6, 1]`
* embedded_text = $[e_{1x100}, e_{2x100}, e_{3x100}, e_{4x100}]$
* $Z = W_{i, 4*100}*e + b$
* $H = softmax(Z)$

So in the above architecture, we pass through a sequence of four words at a time.  Then pass the words through an embedding and concatenate the result to get a vector of 400.  Finally, we pass this through a softmax layer to predict the next word.

Now there are a couple of issues with the above code.  One is that we are defining the linear layer to be a fixed length, where document lengths can vary.  One hundred dimensions for each of the four words in a document.  This is ok if we only need a 4-gram to predict the next word, but what if we'd like to use more words? 

Another issue is that there are separate weights for each word in a document.  So we always multiply the embedding of the first word in the n-gram by the parameters of the first neuron, and the second word by the parameters of the second neuron.  The problem with this is that believe that a lot of the rules that govern the first word also govern the second word, so we'd like to reuse these weights.  And the bigger we make our window, the more weights we need.

### Using an RNN

Now in a recurrent neural network, the approach is the following.  First, we still numericalize our text and translate the word into our embeddings.  
* numericalized_doc = `[1, 10, 20, 6]`
* embedded_text = $[e_{1x100}, e_{2x100}, e_{3x100}, e_{4x100}]$

Now remember our task is to, given the current word and previous words, to predict the next word.  So to use the current word to predict the next one, we simply take the embedded word and multiply it by a weight matrix.

* $W_e \cdot e^t$

And to capture information from the previous words, we use something called a hidden state.  The hidden state is initialized before the first word in our document.

$h_0 = torch.zeros() $

And then the hidden state is multiplied by it's own weight matrix.

$W_h \cdot h^{t -1}$

And updated with each subsequent word according to the following formula:

* $h^{(t)} = \sigma(W_e e^t + W_h h^{t -1} + b)$ 

So we can see that the current hidden state is a combination of a set of weights multiplied by the current word plus a set of weights multiplied by the previous hidden state, plus a bias.

To ultimately predict the next word in a sequence, we can take the last hidden state, multiply by another weight matrix U, and then make our prediction of the next word.

### Seeing it Visually

Let's see this again, in the diagram below.  First looking over to the process on the left, we see that we take a word token, "dog", which is then translated to a one hot vector.  From the one hot vector, we perform vector matrix multiplication to find the related word embedding.  Then comes the process of calculating a hidden state for each word.  

To calculate the hidden state associated with a word, we take the word embedding and multiply it by part of the weight matrix associated with the embedding $W_e$.  And we multiply the previous hidden state (which starts off as a vector of say zeros), and multiply it by a different part of the weight matrix $W_h$.  Then we move onto the next word, and follow the same procedure.  Notice that the weights, $W_e$ and $W_h$ are the same at each step.  And to produce the associated hidden state, $H_t$, only the word embedding $e_t$ and $H_{t -1}$ change.

<img src="./drawio-nn.png" width="70%">

So from above, we can see that we use the function $H_t = F(E_tW_e, H_{t-1}W_h)$ to calculate the hidden state for each word.  The idea is that the $E_tW_e$ tells us information about the current word, and $H_tW_h$ encodes information about all of the previous words in the document.  Then to predict the next word, we can use a separate weight matrix, which is the $softmax(H_tU)$ to predict the next word.  Notice that if we wish, we can do this at each step $t$.

### Summary

In this lesson, we learned about the hypothesis function for a recurrent neural network.  We saw that an RNN has us calculate a hidden state, associated with each word.  And that the hidden state is calculated from the function: $H_t = F(E_tW_t, H_{t-1}W_h)$.  The idea is that $E_tW_t$ encodes information about the current word, and that $H_{t-1}W_h$ encodes information about the preceding words.  We use that information to produce a hidden state H_t, which we can use to predict the next word in the sequence.  Note that at each time step, the same weight matrix is used to produce the next word, and only the hidden state and associated word embedding changes.

### Resources

[RNN Pytorch](https://medium.com/dair-ai/building-rnns-is-fun-with-pytorch-and-google-colab-3903ea9a3a79)